<a href="https://colab.research.google.com/github/RodrigoAgronomia/workshop_esalq/blob/master/python/Draw_Mask_VIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Draw mask from VIA - .csv file

In [1]:
%cd /content
!git clone --depth 1 https://github.com/RodrigoAgronomia/workshop_esalq.git
%cd /content/workshop_esalq
!git pull
%cd /content/workshop_esalq/python

/content
fatal: destination path 'workshop_esalq' already exists and is not an empty directory.
/content/workshop_esalq
Already up to date.
/content/workshop_esalq/python


In [0]:
#Importação das bibliotecas necessárias para rodar o código
%matplotlib inline

import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import re

In [0]:

#Define o mapa de cores para geração da máscara
def colorir(cat):
    h, w = cat.shape[:2]
    msk = np.zeros((h,w,3), dtype = 'uint8')
    msk[cat == 1] = [0,0,255]
    msk[cat == 2] = [128,0,0]
    msk[cat == 3] = [255,0,0]
    msk[cat == 4] = [0,128,0]
    msk[cat == 5] = [0,255,0]
    return(msk)

In [0]:
def get_mask(imgt):
    #Split Image in 3 channels
    blue,green,red = cv2.split(imgt)
    #Normalize values
    green = np.divide(green,255)
    red = np.divide(red,255)
    blue = np.divide(blue,255)

    #Sum of normalized values
    norm = red+green+blue

    r = np.divide(red, norm, out=np.zeros_like(red), where=norm!=0)
    g = np.divide(green, norm, out=np.zeros_like(green), where=norm!=0)
    b = np.divide(blue, norm, out=np.zeros_like(blue), where=norm!=0)
    
    #ExG - ExR
    midx = ((2*g) - r - b) - ((1.4*r) - g)

    #Binarize image
    midx[midx<0] = 0
    midx[midx>0] = 1
    imager = (midx*255).astype('uint8')
    return(imager)

In [0]:
#Definição das pastas necessárias para rodar o script
img_dir = '../data/orig/Corn/'
pts_dir = '../data/seg/pts/'
pts_col_dir = '../data/seg/pts_col/'
anotacao = '../data/anotations.csv'

In [0]:
try:
    os.mkdir('../data/seg/')
    os.mkdir(pts_dir)
    os.mkdir(pts_col_dir)
except:
    pass


In [0]:
#Apresenta as imagens dentro do diretório com o respectivo formato da imagem
img_list = os.listdir(img_dir)
img_list = [v for v in img_list if v.endswith('.JPG')]
print(img_list)

['DJI_0417.JPG', 'DJI_0415.JPG', 'DJI_0394.JPG', 'DJI_0409.JPG', 'DJI_0420.JPG', 'DJI_0416.JPG', 'DJI_0418.JPG', 'DJI_0408.JPG', 'DJI_0402.JPG', 'DJI_0419.JPG', 'DJI_0399.JPG', 'DJI_0405.JPG', 'DJI_0410.JPG', 'DJI_0414.JPG', 'DJI_0400.JPG', 'DJI_0403.JPG', 'DJI_0407.JPG', 'DJI_0412.JPG', 'DJI_0406.JPG', 'DJI_0411.JPG', 'DJI_0393.JPG', 'DJI_0397.JPG', 'DJI_0395.JPG', 'DJI_0421.JPG', 'DJI_0396.JPG', 'DJI_0398.JPG', 'DJI_0404.JPG', 'DJI_0401.JPG', 'DJI_0392.JPG', 'DJI_0413.JPG']


In [0]:
csv_file = pd.read_csv(anotacao,index_col = False) #Faz a leitura do arquivo .csv

for img in img_list:
    lista = csv_file[csv_file.filename == img] #Recebe os dados encontrados dentro do csv para a imagem em questão
    tamanho = lista.shape
    if tamanho[0] != 0:
        imagem = cv2.imread(os.path.join(img_dir, img)) #Carrega a imagem em questão

        msk = np.zeros(imagem.shape[:2], dtype = 'uint8') #Carrega as informações do tamanho da imagem
        # Cria uma mascara auxiliar para o desenho dos circulos ao redor das plantas:
        msk0 = msk.copy()
        msk1 = msk.copy()
        
        # Importa a mascara do indice de vegetacao:
        mskk = get_mask(imagem)
        mskk = (mskk > 100).astype('uint8') 
        msk_name = os.path.join(pts_dir, img.replace('.JPG', '.png')) #Faz a junção do diretório com o nome da imagem, alterando seu formato

    
        for i in range(tamanho[0]):
            line = lista.iloc[i,:] #Recebe todas as marcações realizadas dentro daquela imagem
            region_shape = line.region_shape_attributes #Informa a posição onde o ponto se encontra
            region_attributes = (line.region_attributes) #Informa a classe do ponto
            region_attributes = re.findall('"([^"]*)"', region_attributes) #Faz a separação para encontrar a qual classe pertence o ponto marcado
            coords = [float(s) for s in re.findall(r'-?\d+\.?\d*', region_shape)] #Encontrando valores de x e y
            cx = int(coords[0]) #Coordenadas no eixo X
            cy = int(coords[1]) #Coordenadas no eixo y

            #Define as classes aqui para realizar o desenho da máscara      
            if region_attributes[1] == 'Corn':
                cv2.circle(msk0, (cx,cy), 80, 2, -1)
                cv2.circle(msk1, (cx,cy), 8, 3, -1)
                
                
            if region_attributes[1] == 'Soybean':
                cv2.circle(msk0, (cx,cy), 40, 4, -1)
                cv2.circle(msk1, (cx,cy), 8, 5, -1)
        
        #Faz a combinação das mascaras na máscara principal
        msk[(msk0==2)&(mskk==1)] = 2
        msk[(msk0==4)&(mskk==1)] = 4
        msk[(msk1==3)] = 3
        msk[(msk1==5)] = 5
        msk[(msk==0)&(mskk==1)] = 1
        
                
        print(msk_name) #Apresenta o nome da máscara com o diretório a ser salvo
        cv2.imwrite(msk_name, msk) #Realiza o salvamento do background
    
        col_msk = colorir(msk) #Realiza a coloração da máscara
        col_msk_name = os.path.join(pts_col_dir, img.replace('.JPG', '.png')) #Faz a junção do diretório com o nome da imagem, alterando seu formato
        cv2.imwrite(col_msk_name, col_msk) #Realiza o salvamento da máscara já colorida
        
    else:
        print('A imagem'+' '+img+' '+'não possui anotação no .csv apresentado') #Apresenta uma mensagem caso a imagem lida não apresente informações dentro do .csv

../data/seg/pts/DJI_0417.png
../data/seg/pts/DJI_0415.png
../data/seg/pts/DJI_0394.png
../data/seg/pts/DJI_0409.png
../data/seg/pts/DJI_0420.png
../data/seg/pts/DJI_0416.png
../data/seg/pts/DJI_0418.png
../data/seg/pts/DJI_0408.png
../data/seg/pts/DJI_0402.png
../data/seg/pts/DJI_0419.png
../data/seg/pts/DJI_0399.png
../data/seg/pts/DJI_0405.png
../data/seg/pts/DJI_0410.png
../data/seg/pts/DJI_0414.png
../data/seg/pts/DJI_0400.png
../data/seg/pts/DJI_0403.png
../data/seg/pts/DJI_0407.png
../data/seg/pts/DJI_0412.png
../data/seg/pts/DJI_0406.png
../data/seg/pts/DJI_0411.png
../data/seg/pts/DJI_0393.png
../data/seg/pts/DJI_0397.png
../data/seg/pts/DJI_0395.png
../data/seg/pts/DJI_0421.png
../data/seg/pts/DJI_0396.png
../data/seg/pts/DJI_0398.png
../data/seg/pts/DJI_0404.png
../data/seg/pts/DJI_0401.png
../data/seg/pts/DJI_0392.png
../data/seg/pts/DJI_0413.png
